# To Do List:


Structure:
- [ ] General refactoring and packaging
- [ ] partial functions

Additions:
- [ ] Financial Calculator

Analysis:
- [ ] wRVU
- [ ] Waiting Room function
- [ ] LWBS function

ML:
- [ ] Patient Inflow Predictive Analytics

## Load data and dependencies

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive,interact_manual, fixed, HBox, Label, VBox, GridspecLayout, Layout

# import os
# from scipy.stats import poisson
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
# %matplotlib inline

In [2]:
# Load CSVs
path = './data/'
provider_profiles = pd.read_csv(path+'pp_contacts decisions.csv', index_col='Provider Profiles')
arrivals = pd.read_csv(path+'dp_arrivals.csv')
other = pd.read_csv(path+'dp_other.csv')
rooms = pd.read_csv(path+'dp_rooms.csv')
wRVU = pd.read_csv(path+'dp_wRVU.csv')

arrivals['Date'].fillna(value='Week Average', axis=0, inplace=True)
rooms['Date'].fillna(value='Week Average', axis=0, inplace=True)

# Hourly Table

### Hourly Table Variables

In [4]:
times_24hr_clck = [str(i)+':00' for i in range(0,24)]

times = ['12:00:00 AM', '01:00:00 AM', '02:00:00 AM', '03:00:00 AM', '04:00:00 AM', '05:00:00 AM', 
         '06:00:00 AM', '07:00:00 AM', '08:00:00 AM','09:00:00 AM', '10:00:00 AM', '11:00:00 AM', 
         '12:00:00 PM','01:00:00 PM', '02:00:00 PM', '03:00:00 PM', '04:00:00 PM','05:00:00 PM', 
         '06:00:00 PM', '07:00:00 PM', '08:00:00 PM','09:00:00 PM', '10:00:00 PM', '11:00:00 PM']

### Hourly Table Secondary Functions

In [5]:
def get_provider_profiles(num_staff):
    contacts = np.zeros((num_staff, 24))

    for i in range(0, num_staff+1):
        try:
            contacts[i,:] = provider_profiles.loc[providers[i].value,:].values[:24]
        except: 
            pass
    return contacts

In [6]:
def provider_number_sparse(num_staff):

    providers_ph = np.zeros((num_staff, 24))
    for i in range(0, num_staff):
        z = [0]*24

        start = int(start_time[i].get_interact_value().split(':')[0])
        end = int(end_time[i].get_interact_value().split(':')[0])

        if start<end:
            z[start:end] = [1]*(end-start)
        else:
            z[start:] = [1]*(24-start)
            z[:end] = [1]*end

        providers_ph[i]= z
        
    return providers_ph

In [7]:
def providers_aligned_shifts(num_staff):
    pp = get_provider_profiles(staff)
    num_prov_hour_sparse = provider_number_sparse(staff)

    for i in range(0, num_staff):
        if sum(num_prov_hour_sparse[i]) != 0:
            if (num_prov_hour_sparse[i][0] == 1.) & (num_prov_hour_sparse[i][-1] == 1.):
                tot = int(sum(num_prov_hour_sparse[i]))
                first_zero = np.argmin(num_prov_hour_sparse[i])
                first_one = np.argmax(num_prov_hour_sparse[i][first_zero:]) + first_zero
                num_prov_hour_sparse[i][first_one:] = pp[i][:24-first_one]
                num_prov_hour_sparse[i][:first_zero] = pp[i][:tot][-first_zero:]
            
            elif (num_prov_hour_sparse[i][-1] == 1.) & (not (num_prov_hour_sparse[i][0] == 1.)):
                start = np.argmax(num_prov_hour_sparse[i])
                end = start+int(sum(num_prov_hour_sparse[i]))        
                num_prov_hour_sparse[i][start:end+1] = pp[i][:end-start] * num_prov_hour_sparse[i][start:end+1]

            else: 
                start = np.argmax(num_prov_hour_sparse[i])
                end = start+int(sum(num_prov_hour_sparse[i]))        
                num_prov_hour_sparse[i][start:end+1] = pp[i][:end-start+1] * num_prov_hour_sparse[i][start:end+1]
    
    return num_prov_hour_sparse

In [8]:
def provider_room_cycle_time_aligned(num_staff):
    scheduled_providers = []
    providers_room_cycle = np.zeros((num_staff, 24))

    for i in range(0,num_staff):
        scheduled_providers.append(providers[i].get_interact_value())

    total_provider_time = []

    for staffer in scheduled_providers:
        a = provider_profiles.loc[staffer]['Average of Arrival to Provider Hrs']
        b = provider_profiles.loc[staffer]['Average of Provider to Decision Hrs']
        total_provider_time.append(a + b)

    a = provider_number_sparse(num_staff)

    for b in range(0, num_staff):
        providers_room_cycle[b] = np.where(a[b]!=0, total_provider_time[b], 0)

    return providers_room_cycle

In [9]:
def admit_boarding_cycle_time_aligned(num_staff):
    provider_hour = provider_number_sparse(num_staff)
    dept_val = dept_profiles.get_interact_value()
    dept_admit_rate = float(other[other['Department Profile'] == dept_val]['Admit Rate'].tolist()[0].split('%')[0])
    dept_admit_dec_depart = other[other['Department Profile'] == dept_val]['Admit Decision to Depart'].tolist()[0]
    val = (dept_admit_rate/100) * (admit_boarding_time.get_interact_value() + dept_admit_dec_depart)

    for b in range(0, num_staff):
            provider_hour[b] = np.where(provider_hour[b]!=0, val, 0)

    return provider_hour

In [10]:
def discharge_boarding_cycle_time(num_staff):
    provider_hour = provider_number_sparse(num_staff)
    dept_val = dept_profiles.get_interact_value()
    dept_discharge_rate = 1-((float(other[other['Department Profile'] == dept_val]['Admit Rate'].tolist()[0].split('%')[0]))/100)
    dept_discharge_dec_depart = other[other['Department Profile'] == dept_val]['DC Decision to Depart'].tolist()[0]
    val = dept_discharge_rate * (discharge_boarding_time.get_interact_value() + dept_discharge_dec_depart)

    for b in range(0, num_staff):
            provider_hour[b] = np.where(provider_hour[b]!=0, val, 0)

    return provider_hour

In [11]:
# Admit % x Admit Cycle + Discharge % x Discharge Cycle aligned with shifts
def aa_dd(num_staff):
    p_admit_p = []
    p_discharge_p = []
    for i in range(0, num_staff):
        p = providers[i].get_interact_value()
        p_admit_p.append(provider_profiles['Admit %'].loc[p])
        p_discharge_p.append(provider_profiles['Discharge %'].loc[p])

    pap = np.array([0 if str(x) == 'nan' else x for x in p_admit_p])
    pdp = np.array([1 if str(x) == 'nan' else x for x in p_discharge_p])

    dbst = discharge_boarding_cycle_time(num_staff)
    abcta = admit_boarding_cycle_time_aligned(num_staff)

    return (abcta.T*pap).T + (dbst.T*pdp).T

In [12]:
def total_room_cycle_time(num_staff):
    a = provider_room_cycle_time_aligned(num_staff)
    b = aa_dd(num_staff)
    return a+b

In [13]:
def profiles_decisions_percentage(num_staff):

    p_total_decisions=np.zeros((num_staff,24))

    for i in range(0, num_staff):
        p = providers[i].get_interact_value()
        decisions_per_hour = provider_profiles.loc[p][25:49].values
        total = sum(decisions_per_hour)
        p_total_decisions[i] = decisions_per_hour / total

    return p_total_decisions

In [14]:
def decision_percent_aligned(num_staff):
    pp = profiles_decisions_percentage(num_staff)
    num_prov_hour_sparse = provider_number_sparse(num_staff)

    for i in range(0, num_staff):
        if sum(num_prov_hour_sparse[i]) != 0:
            if (num_prov_hour_sparse[i][0] == 1.) & (num_prov_hour_sparse[i][-1] == 1.):
                tot = int(sum(num_prov_hour_sparse[i]))
                first_zero = np.argmin(num_prov_hour_sparse[i])
                first_one = np.argmax(num_prov_hour_sparse[i][first_zero:]) + first_zero
                num_prov_hour_sparse[i][first_one:] = pp[i][:24-first_one]
                num_prov_hour_sparse[i][:first_zero] = pp[i][:tot][-first_zero:]

            elif (num_prov_hour_sparse[i][-1] == 1.) & (not (num_prov_hour_sparse[i][0] == 1.)):
                start = np.argmax(num_prov_hour_sparse[i])
                end = start+int(sum(num_prov_hour_sparse[i]))        
                num_prov_hour_sparse[i][start:end+1] = pp[i][:end-start] * num_prov_hour_sparse[i][start:end+1]

            else: 
                start = np.argmax(num_prov_hour_sparse[i])
                end = start+int(sum(num_prov_hour_sparse[i]))        
                num_prov_hour_sparse[i][start:end+1] = pp[i][:end-start+1] * num_prov_hour_sparse[i][start:end+1]

    return num_prov_hour_sparse

In [15]:
# Avg Weighted Room Cycle Time
def avg_weighted_room_cycle(num_staff):
    a = total_room_cycle_time(num_staff)
    b = decision_percent_aligned(num_staff)
    return np.sum(a * b, axis=0) / np.sum(b, axis=0)

In [16]:
# Bed Capacity Calculations
def bed_capacity_calc(num_staff):
   
    avg = avg_weighted_room_cycle(staff)
    patients = get_arrivals()
    res = np.zeros((10, 24))
    res[0] = np.where(avg>=1, patients, patients*avg)

    for i in range(1, 10):
        for j in range(24):
            if avg[j] >= i+1:
                res[i,j] = res[i-1,j-1]
            elif avg[j] >= i:
                res[i,j] = (avg[j]-(i)) * res[i-1,j-1]
            else:
                res[i,j] = 0 
    
    return np.round(res, decimals=2)


In [17]:
def total_additional_seen(num_staff):
    beds = bed_availability(num_staff)
    total = total_waiting_room(num_staff)
    twmabs = np.where((total - beds)<0, 0, (total - beds))
    return total - twmabs

### Hourly Table Primary Functions

In [18]:
def get_arrivals(*args, event=None):
    dept = dept_profiles.get_interact_value()
    weekday = day_week.get_interact_value()
    if not event:
        return arrivals[(arrivals['Department Profile'] == dept) & (arrivals['Date'] == weekday)  & (arrivals['Event'] != 'Summerfest')][times].values

In [19]:
def get_beds(*args, event=None):
    dept = dept_profiles.get_interact_value()
    weekday = day_week.get_interact_value()
    if not event:
        return rooms[(rooms['Department Profile'] == dept) & (rooms['Date'] == weekday)  & (rooms['Event'] != 'Summerfest')][times].values

In [20]:
def get_docs(num_staff,*args):
    docs=[]
    for i in range(num_staff):
        if position[i].get_interact_value() == 'Doc': docs.append(i)

    return np.sum(provider_number_sparse(staff)[docs,:],axis=0)

In [21]:
def get_apps(num_staff,*args):
    apps=[]
    for i in range(num_staff):
        if position[i].get_interact_value() == 'APP':apps.append(i)
    return np.sum(provider_number_sparse(staff)[apps,:],axis=0)    

In [22]:
def get_capacity(num_staff, *args):
    return np.sum(providers_aligned_shifts(num_staff),axis=0).tolist()

In [23]:
def get_providerutil(*args):
    try:
        res = np.round(get_arrivals() / get_capacity(staff), decimals=3)
    
    except ZeroDivisionError:
        print('Provider Utilization: division by zero')
    
    return res

In [24]:
def get_bednec(num_staff, *args):
    beds = bed_capacity_calc(num_staff)
    tot_unseen = 0 
    return np.sum(beds, axis=0)

In [25]:
def get_bedutil(num_staff, *args):
    num_beds = get_beds()
    used_beds = get_bednec(num_staff)
    return np.round(used_beds / num_beds, decimals=3)

In [26]:
def get_nurses(num_staff, *args):
    used_beds = get_bednec(num_staff)
    return np.ceil((used_beds/4) +2)

In [27]:
def get_waitingroom(num_staff, *args):
    available_beds = bed_availability(staff)
    beds = np.where(available_beds<0, 0, available_beds)
    waiting = waiting_room1(num_staff)
    total = total_waiting_room(num_staff)
    res = total - beds
    return np.ceil(np.where(res<0, 0, res))

In [28]:
def bed_availability(num_staff, *args):
    num_beds = get_beds()
    used_beds = get_bednec(num_staff)
    return np.clip((num_beds - used_beds), 0, 1000)

In [29]:
def waiting_room1(num_staff, *args):
    num_beds = get_beds()
    bed_nec = get_bednec(num_staff)
    return np.where(bed_nec>num_beds, bed_nec-num_beds, 0)

In [30]:
def total_waiting_room(num_staff, *args):
    waiting1 = waiting_room1(num_staff)
    res = np.zeros((5, 24))
    res[0] = waiting1
    for x in range(0,24):
        for i in range(1,5):
            if i == 1:
                res[1, x] = res[0,x-1]*.75
            elif i == 2:
                res[2, x] = res[0,x-2]*.50
            elif i == 3:
                res[3, x] = res[0,x-3]*.38
            else:
                res[4, x] = res[0,x-4]*.35

    return np.sum(res, axis=0)

In [31]:
def get_lwbs(num_staff, *args):
    total = total_additional_seen(num_staff)
    res = np.zeros((5, 24))
    res[0] = waiting_room1(num_staff)
    for x in range(0,24):
        for i in range(1,5):
            if i == 1:
                res[1, x] = res[0,x-1]*.25
            elif i == 2:
                res[2, x] = res[0,x-2]*.25
            elif i == 3:
                res[3, x] = res[0,x-3]*.12
            else:
                res[4, x] = res[0,x-4]*.03
    res[0] = [0]*24
    lwbs = np.round(np.sum(res, axis=0), decimals=2)
    final = lwbs - total
    
    return np.ceil(np.where(final<0, 0, final))

__Generate Table__

In [32]:
hourly_metrics = ['# of Patient Arrivals', '# of Beds', '# of Docs', '# of APPs', 'Provider Capacity', 
                  'Bed Necessity', 'Provider Utilization', 'Bed Utilization', 'Waiting Room (2.0)', 'LWBS',
                  'Nurses needed']

callbacks = {'# of Patient Arrivals': get_arrivals, '# of Beds': get_beds, '# of Docs': get_docs, '# of APPs': get_apps, 
             'Provider Capacity': get_capacity, 'Bed Necessity': get_bednec, 'Provider Utilization': get_providerutil, 
             'Bed Utilization': get_bedutil, 'Waiting Room (2.0)': get_waitingroom, 'LWBS': get_lwbs, 
             'Nurses needed': get_nurses}

In [33]:
def util_colors(val):
    if val <= 90.:
        color = 'blue'
    elif val >= 91. and val < 110.:
        color = 'green'
    elif val >= 110. and val < 120.:
        color = 'orange'
    else:
        color = 'red'
    return 'color: %s' % color

def wait_colors(val):
    if val == 0:
        color = 'green'
    else:
        color = 'red'
    return 'color: %s' % color

In [34]:
def generate_table(num_staff, *args):
    df = pd.DataFrame(index=hourly_metrics, columns=times_24hr_clck)
    for i in hourly_metrics:
        df.loc[i] = callbacks.get(i)(num_staff)
    
    df.loc['Provider Utilization'] = (df.loc['Provider Utilization'] * 100)
    df.loc['Bed Utilization'] = (df.loc['Bed Utilization'] * 100)
    
    s = df.style.applymap(util_colors, subset=pd.IndexSlice[['Provider Utilization','Bed Utilization'], :])\
                .applymap(wait_colors, subset=pd.IndexSlice[['Waiting Room (2.0)'], :] )
#                 .format({'Provider Utilization': '{:,.2f}'.format, 'Bed Utilization': '{:,.2f}'.format })
    
    return s

In [ ]:
# Set CSS properties for th elements in dataframe
# th_props = [
#   ('font-size', '11px'),
#   ('text-align', 'center'),
#   ('font-weight', 'bold'),
#   ('color', '#6d6d6d'),
#   ('background-color', '#f7f7f9')
#   ]

# # Set CSS properties for td elements in dataframe
# td_props = [
#   ('font-size', '11px')
#   ]

# # Set table styles
# styles = [
#   dict(selector="th", props=th_props),
#   dict(selector="td", props=td_props)
#   ]


# (df.style
#     .applymap(color_negative_red, subset=['total_amt_usd_diff','total_amt_usd_pct_diff'])
#     .format({'total_amt_usd_pct_diff': "{:.2%}"})
#     .set_table_styles(styles))

# Schedule Grid

### Schedule Options

In [35]:
doc_options = ['D - Doc - 2018','D - Doc - A','D - Doc - B','D - Doc - C','D - Doc - D','D - Doc - E','D - Doc - F',
               'D - Doc - G','D - Doc - H','D - Doc - I','D - Doc - J','D - Doc - K','D - Doc - L','D - Doc - M','D - Doc - N',
               'D - Doc - O','D - Doc - P','D - Doc - Q','D - Doc - R','D - Doc - S','D - Doc - T','D - Doc - U','D - Doc - V',
               'D - Doc - W','D - Doc - X','D - Doc - Y','D - Doc - Z','D - Doc - Za','D - Doc - Zb','D - Doc - Zc',
               'D - Doc - Zd','D - Doc - Ze','D - Doc - Zf','D - Doc - Zg','D - Doc - Zh', 'D']

APP_options = ['D - APP - /Robot','D - APP - 2018','D - APP - A','D - APP - B','D - APP - C','D - APP - D',
               'D - APP - E','D - APP - F','D - APP - G','D - APP - H','D - APP - I','D - APP - J','D - APP - K',
               'D - APP - L','D - APP - M','D - APP - N','D - APP - O','D - APP - P','D - APP - Q','D - APP - R','D - APP - S',
               'D - APP - T','D - APP - U','D - APP - V','D - APP - W','D - APP - X','D - APP - Y','D - APP - Z',
               'D - APP - Za', 'D - Intake','D - Obs','D - Quad A','D - Test 1','D - Test 2','D - Test 3','D - Test 4',]

In [36]:
presets = [['Doc', '0:00', '6:00', 'Main ED', 'D - Doc - A'],
 ['Doc', '2:00', '8:00', 'Main ED', 'D - Doc - B'],
 ['Doc', '4:00', '12:00', 'Main ED', 'D - Doc - C'],
 ['Doc', '8:00', '16:00', 'Main ED', 'D - Doc - D'],
 ['Doc', '16:00', '4:00', 'Main ED', 'D - Doc - E'],
 ['Doc', '9:00', '0:00', 'Main ED', 'D - Doc - F'],
 ['APP', '0:00', '8:00', 'Obs', 'D - Obs'],
 ['APP', '12:00', '23:00', 'Obs', 'D - Obs'],
 ['APP', '0:00', '8:00', 'Quad A', 'D - APP - A'],
 ['APP', '7:00', '18:00', 'Quad A', 'D - APP - B'],
 ['APP', '0:00', '12:00', 'Intake', 'D - Intake'],
 ['APP', '12:00', '23:00', 'Main ED', 'D - APP - C'],
 ['APP', '16:00', '4:00', 'Main ED', 'D - APP - D']]

In [37]:
style = {'description_width': 'initial'}
times_24hr_clck = [str(i)+':00' for i in range(0,24)]

### Scheduler Functions

In [38]:
def update_provider_profile(change,*args):
    mid = change['owner'].model_id
    pp = 0
    for i,d in position_ids:
        if d == mid:
            pp = i
    
    if change['new'] == 'Doc':
        providers[pp].options = doc_options 
    elif change['new'] == 'APP':
        providers[pp].options = APP_options
    else:
        pass
 

In [39]:
def start_total_hours(change, *args):
    modelid = change['owner'].model_id
    pp = 0

    for i,d in start_ids:
        if d == modelid:
            pp = i
    
    start = int(change['new'].split(':')[0])
    end = int(end_time[pp].value.split(':')[0])
    
    if start < end:
        total_hours[pp].value = end-start
    else:
        total_hours[pp].value = (24-start) + end

In [40]:
def end_total_hours(change, *args):
    modelid = change['owner'].model_id
    pp = 0

    for i,d in end_ids:
        if d == modelid:
            pp = i
    
    end = int(change['new'].split(':')[0])
    start = int(start_time[pp].value.split(':')[0])
    
    if start < end:
        total_hours[pp].value = end-start
    else:
        total_hours[pp].value = (24-start) + end

In [41]:
# initial Admit Boarding Time
def initial_admit_boarding_time():
    a = other[other['Department Profile'] == dept_profiles.get_interact_value()]['Admit Decision to Depart'].tolist()[0]
    if str(a) == 'nan': a=0
    b = int(admit_boarding_time.get_interact_value())
    return  a+b

In [42]:
# initial Discharge Boarding Time
def initial_discharge_boarding_time():
    a = other[other['Department Profile'] == dept_profiles.get_interact_value()]['DC Decision to Depart'].tolist()[0]
    if str(a) == 'nan': a=0
    b = int(discharge_boarding_time.get_interact_value())
    return  a+b 

In [43]:
def update_admit_boarding_time_dept(change):
    new_dept = other[other['Department Profile'] == change['new']]['Admit Decision to Depart'].tolist()[0]
    admit_val = int(admit_boarding_time.get_interact_value())
    if str(new_dept) == 'nan': new_dept = 0 
    admit_boarding_time_value.value = new_dept + admit_val

In [44]:
def update_admit_boarding_time(change):
    new_admit_val = int(change['new'])
    decision_by_dept = other[other['Department Profile'] == dept_profiles.get_interact_value()]['Admit Decision to Depart'].tolist()[0]
    admit_boarding_time_value.value = new_admit_val + decision_by_dept

In [45]:
def update_discharge_boarding_time_dept(change):
    new_dept = other[other['Department Profile'] == change['new']]['DC Decision to Depart'].tolist()[0]
    new_discharge_val = int(discharge_boarding_time.get_interact_value())
    if str(new_dept) == 'nan': new_dept = 0 
    discharge_boarding_time_value.value = new_dept + new_discharge_val

In [46]:
def update_discharge_boarding_time(change):
    new_discharge_val = int(change['new'])
    decision_by_dept = other[other['Department Profile'] == dept_profiles.get_interact_value()]['DC Decision to Depart'].tolist()[0]
    if str(decision_by_dept) == 'nan': decision_by_dept = 0
    discharge_boarding_time_value.value = new_discharge_val + decision_by_dept

### Scheduler

In [47]:
# Set Docs and Advanced practice providers (APPs)
docs=6
apps=7
staff = docs+apps

In [48]:
# create variable widgets
dept_profiles = widgets.Dropdown(options=arrivals['Department Profile'].unique().tolist(), layout=Layout(height='auto', width='auto'))
day_week = widgets.Dropdown(options=arrivals['Date'].unique().tolist(), layout=Layout(height='auto', width='auto'))
admit = widgets.IntSlider(value=100, min=-200, max=200, step=10, layout=Layout(height='auto', width='auto'))
admit_boarding_time = widgets.IntText(value=6, layout=Layout(height='auto', width='auto'))
admit_boarding_time_value = widgets.FloatText(initial_admit_boarding_time(), layout=Layout(height='auto', width='auto'))
discharge_boarding_time = widgets.FloatText(value=0, layout=Layout(height='auto', width='auto'))
discharge_boarding_time_value = widgets.FloatText(initial_discharge_boarding_time(), layout=Layout(height='auto', width='auto'))
daily_volume = widgets.IntSlider(value=100, min=-200, max=200, step=10, layout=Layout(height='auto', width='auto'))

# create variable labels
dept_profiles_button = widgets.Button(description='Dept Profile:', disabled=True, layout=Layout(height='auto', width='auto'))
day_week_button = widgets.Button(description='Date:', disabled=True, layout=Layout(height='auto', width='auto'))
admit_button = widgets.Button(description='Admit %:', disabled=True, layout=Layout(height='auto', width='auto'))
daily_volume_button = widgets.Button(description='Daily Volume %:', disabled=True,button_style='',tooltip='',icon='', layout=Layout(height='auto', width='auto'))
admit_boarding_time_button = widgets.Button(description='Admit Boarding Time:', disabled=True,button_style='',tooltip='',icon='', layout=Layout(height='auto', width='auto'))
discharge_boarding_time_button = widgets.Button(description='Discharge Boarding Time:', disabled=True,button_style='',tooltip='',icon='', layout=Layout(height='auto', width='auto'))

# create headers 
header_text = ['Shift', 'Position', 'Start Time', 'End Time', 'Role', 'Provider Profile', 'Total Hours']
headers = [widgets.Button(description=y, disabled=True, layout=Layout(height='auto', width='auto')) for y in header_text]


# create schedule widgets
pro_pro = provider_profiles.index.values.tolist()
providers = [widgets.Dropdown(options=pro_pro , layout=Layout(height='auto', width='auto')) for x in range(0, staff+1)]
active = [widgets.Checkbox(value=False) for x in range(0, staff+1)]
start_time =  [widgets.Dropdown(options=times_24hr_clck, layout=Layout(height='auto', width='auto')) for x in range(0,staff+1)]
end_time = [widgets.Dropdown(options=times_24hr_clck, layout=Layout(height='auto', width='auto')) for x in range(0,staff+1)]
position = [widgets.Dropdown(options=['--', 'Doc', 'APP'],layout=Layout(height='auto', width='auto')) for x in range(0,staff+1)]
role = [widgets.Dropdown(options=['--','Main ED', 'Obs', 'Quad A', 'Intake'], layout=Layout(height='auto', width='auto')) for x in range(0, staff+1)]
shift = [widgets.Button(description=y, disabled=True,button_style='',tooltip='',icon='', layout=Layout(height='auto', width='auto')) for y in [str(x) for x in range(1, staff+1)]]
total_hours = [widgets.IntText(0, layout=Layout(height='auto', width='auto')) for y in range(1, staff+1)]


In [49]:
b = 6
grid_rows = staff + b + 2

# table_layout = {'width':'auto', 'height':'auto'}
grid = GridspecLayout(grid_rows, 7)

In [50]:
# add variable widgets to grid
grid[0, 0] = dept_profiles_button
grid[0, 1] = dept_profiles
grid[1, 0] = day_week_button
grid[1, 1] = day_week
grid[0, 2:4] = admit_button
grid[0, 4:6] = admit
grid[3, 2:4] = daily_volume_button
grid[3, 4:6] = daily_volume
grid[1, 2:4] = admit_boarding_time_button
grid[1, 4] = admit_boarding_time
grid[1, 5] = admit_boarding_time_value
grid[2, 2:4] = discharge_boarding_time_button
grid[2, 4] = discharge_boarding_time
grid[2, 5] = discharge_boarding_time_value


# add headers to grid
for i in range(0, 7):
    grid[b-1,i] = headers[i]


# get observe ids
position_ids = [(indx, wid.model_id) for indx, wid in enumerate(position)]
start_ids = [(indx, wid.model_id) for indx, wid in enumerate(start_time)]
end_ids = [(indx, wid.model_id) for indx, wid in enumerate(end_time)]

# add schedule widgets to grid
for i in range(b, grid_rows-2):
    grid[i, 0] = shift[i-b]
    grid[i, 1] = position[i-b]
    grid[i, 2] = start_time[i-b]
    grid[i, 3] = end_time[i-b]
    grid[i, 4] = role[i-b]
    grid[i, 5] = providers[i-b]
    grid[i, 6] = total_hours[i-b]

# link widgets
for i in range(0, staff):
    position[i].observe(update_provider_profile, names='value')
    start_time[i].observe(start_total_hours, names='value')
    end_time[i].observe(end_total_hours, names='value')


dept_profiles.observe(update_admit_boarding_time_dept, names='value')
admit_boarding_time.observe(update_admit_boarding_time, names='value')
dept_profiles.observe(update_discharge_boarding_time_dept, names='value')
discharge_boarding_time.observe(update_discharge_boarding_time, names='value')

# Simulator

In [51]:
# presets for grid
def load_presets(num_staff, presets):
    for i in range(0, num_staff):
        position[i].value = presets[i][0]
        start_time[i].value = presets[i][1]
        end_time[i].value = presets[i][2]
        role[i].value = presets[i][3]
        providers[i].value = presets[i][4]

In [53]:
load_presets(staff, presets)

In [52]:
grid

GridspecLayout(children=(Button(description='Dept Profile:', disabled=True, layout=Layout(grid_area='widget001…

In [56]:
generate_table(staff)

,0:00,1:00,2:00,3:00,4:00,5:00,6:00,7:00,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00
# of Patient Arrivals,2.2,2,1.7,1.6,1.5,1.3,1.2,2.1,2.4,3.7,4.2,4.7,4.7,4.2,5,4.7,5.2,5.7,5.5,5.7,4.6,4.1,3.3,2.7
# of Beds,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
# of Docs,2,2,3,3,3,3,2,2,2,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2
# of APPs,4,4,4,4,3,3,3,4,2,2,2,2,3,3,3,3,4,4,3,3,3,3,3,1
Provider Capacity,10.826,7.768,10.408,9.168,12.377,11.938,7.584,10.64,9.623,10.058,11.543,7.62,10.629,9.289,7.003,6.376,12.617,10.714,6.722,7.365,4.665,6.444,3.854,1.333
Bed Necessity,9.17,6.4,5.96,5.47,6.02,5.31,4.74,5.28,5.83,8.53,12.73,15.16,17.61,18.19,18.87,19.21,22.28,22.62,20.83,22.05,22.48,21.18,18.59,18.64
Provider Utilization,20.3,25.7,16.3,17.5,12.1,10.9,15.8,19.7,24.9,36.8,36.4,61.7,44.2,45.2,71.4,73.7,41.2,53.2,81.8,77.4,98.6,63.6,85.6,202.6
Bed Utilization,50.9,35.6,33.1,30.4,33.4,29.5,26.3,29.3,32.4,47.4,70.7,84.2,97.8,101.1,104.8,106.7,123.8,125.7,115.7,122.5,124.9,117.7,103.3,103.6
Waiting Room (2.0),0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,6,9,10,11,13,12,8,6
LWBS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,3,3,3,3,3,2
